In [93]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [102]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
submission= pd.read_csv("SampleSubmission.csv")

In [12]:
train_df

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
0,DRA12,BABATUNJI010,DRA12_BABATUNJI010,11.60,Low Sugar,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08
1,DRA12,BABATUNJI013,DRA12_BABATUNJI013,11.60,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
2,DRA12,BABATUNJI017,DRA12_BABATUNJI017,11.60,Low Sugar,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69
3,DRA12,BABATUNJI018,DRA12_BABATUNJI018,11.60,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
4,DRA12,BABATUNJI035,DRA12_BABATUNJI035,11.60,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,NCZ54,BABATUNJI019,NCZ54_BABATUNJI019,NaN,Low Sugar,0.145952,Household,402.39,1992,Small,Cluster 1,Grocery Store,406.14
4986,NCZ54,BABATUNJI027,NCZ54_BABATUNJI027,NaN,Low Sugar,0.082956,Household,410.14,1992,Medium,Cluster 3,Supermarket Type3,13808.69
4987,NCZ54,BABATUNJI045,NCZ54_BABATUNJI045,14.65,Low Sugar,0.083528,Household,406.14,2009,NaN,Cluster 2,Supermarket Type1,5685.93
4988,NCZ54,BABATUNJI046,NCZ54_BABATUNJI046,14.65,Low Sugar,0.083359,Household,404.89,2004,Small,Cluster 1,Supermarket Type1,11778.00


In [13]:
test_df.head()

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type
0,DRA59,BABATUNJI010,DRA59_BABATUNJI010,8.270,Normal Sugar,0.214125,Soft Drinks,459.98,2005,NaN,Cluster 3,Grocery Store
1,DRA59,BABATUNJI013,DRA59_BABATUNJI013,8.270,Normal Sugar,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1
2,DRB01,BABATUNJI013,DRB01_BABATUNJI013,7.390,Low Sugar,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1
3,DRB13,BABATUNJI010,DRB13_BABATUNJI010,6.115,Normal Sugar,0.011791,Soft Drinks,472.63,2005,NaN,Cluster 3,Grocery Store
4,DRB13,BABATUNJI013,DRB13_BABATUNJI013,6.115,Normal Sugar,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990 entries, 0 to 4989
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Item_ID              4990 non-null   object 
 1   Store_ID             4990 non-null   object 
 2   Item_Store_ID        4990 non-null   object 
 3   Item_Weight          4188 non-null   float64
 4   Item_Sugar_Content   4990 non-null   object 
 5   Item_Visibility      4990 non-null   float64
 6   Item_Type            4990 non-null   object 
 7   Item_Price           4990 non-null   float64
 8   Store_Start_Year     4990 non-null   int64  
 9   Store_Size           3540 non-null   object 
 10  Store_Location_Type  4990 non-null   object 
 11  Store_Type           4990 non-null   object 
 12  Item_Store_Returns   4990 non-null   float64
dtypes: float64(4), int64(1), object(8)
memory usage: 506.9+ KB


In [15]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3532 entries, 0 to 3531
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Item_ID              3532 non-null   object 
 1   Store_ID             3532 non-null   object 
 2   Item_Store_ID        3532 non-null   object 
 3   Item_Weight          2871 non-null   float64
 4   Item_Sugar_Content   3532 non-null   object 
 5   Item_Visibility      3532 non-null   float64
 6   Item_Type            3532 non-null   object 
 7   Item_Price           3532 non-null   float64
 8   Store_Start_Year     3532 non-null   int64  
 9   Store_Size           2573 non-null   object 
 10  Store_Location_Type  3532 non-null   object 
 11  Store_Type           3532 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 331.3+ KB


In [16]:
missing_percentage_train=train_df.isnull().sum()/len(train_df)*100
missing_percentage_train

Item_ID                 0.000000
Store_ID                0.000000
Item_Store_ID           0.000000
Item_Weight            16.072144
Item_Sugar_Content      0.000000
Item_Visibility         0.000000
Item_Type               0.000000
Item_Price              0.000000
Store_Start_Year        0.000000
Store_Size             29.058116
Store_Location_Type     0.000000
Store_Type              0.000000
Item_Store_Returns      0.000000
dtype: float64

In [17]:
missing_percentage_test=test_df.isnull().sum()/len(test_df)*100
missing_percentage_test

Item_ID                 0.000000
Store_ID                0.000000
Item_Store_ID           0.000000
Item_Weight            18.714609
Item_Sugar_Content      0.000000
Item_Visibility         0.000000
Item_Type               0.000000
Item_Price              0.000000
Store_Start_Year        0.000000
Store_Size             27.151755
Store_Location_Type     0.000000
Store_Type              0.000000
dtype: float64

In [18]:
train_df['Item_Weight'].fillna(train_df['Item_Weight'].mean(), inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_236\1036376471.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Item_Weight'].fillna(train_df['Item_Weight'].mean(), inplace=True)


In [19]:
train_df['Store_Size'].fillna(train_df['Store_Size'].mode()[0], inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_236\721585175.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Store_Size'].fillna(train_df['Store_Size'].mode()[0], inplace=True)


In [20]:
train_df

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
0,DRA12,BABATUNJI010,DRA12_BABATUNJI010,11.600000,Low Sugar,0.068535,Soft Drinks,357.54,2005,Medium,Cluster 3,Grocery Store,709.08
1,DRA12,BABATUNJI013,DRA12_BABATUNJI013,11.600000,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
2,DRA12,BABATUNJI017,DRA12_BABATUNJI017,11.600000,Low Sugar,0.041178,Soft Drinks,350.79,2014,Medium,Cluster 2,Supermarket Type1,6381.69
3,DRA12,BABATUNJI018,DRA12_BABATUNJI018,11.600000,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
4,DRA12,BABATUNJI035,DRA12_BABATUNJI035,11.600000,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,NCZ54,BABATUNJI019,NCZ54_BABATUNJI019,12.908838,Low Sugar,0.145952,Household,402.39,1992,Small,Cluster 1,Grocery Store,406.14
4986,NCZ54,BABATUNJI027,NCZ54_BABATUNJI027,12.908838,Low Sugar,0.082956,Household,410.14,1992,Medium,Cluster 3,Supermarket Type3,13808.69
4987,NCZ54,BABATUNJI045,NCZ54_BABATUNJI045,14.650000,Low Sugar,0.083528,Household,406.14,2009,Medium,Cluster 2,Supermarket Type1,5685.93
4988,NCZ54,BABATUNJI046,NCZ54_BABATUNJI046,14.650000,Low Sugar,0.083359,Household,404.89,2004,Small,Cluster 1,Supermarket Type1,11778.00


In [21]:
test_df['Item_Weight'].fillna(test_df['Item_Weight'].mean(), inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_236\279443694.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Item_Weight'].fillna(test_df['Item_Weight'].mean(), inplace=True)


In [22]:
test_df['Store_Size'].fillna(test_df['Store_Size'].mode()[0], inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_236\2513149234.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Store_Size'].fillna(test_df['Store_Size'].mode()[0], inplace=True)


In [23]:
test_df

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type
0,DRA59,BABATUNJI010,DRA59_BABATUNJI010,8.270,Normal Sugar,0.214125,Soft Drinks,459.98,2005,Medium,Cluster 3,Grocery Store
1,DRA59,BABATUNJI013,DRA59_BABATUNJI013,8.270,Normal Sugar,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1
2,DRB01,BABATUNJI013,DRB01_BABATUNJI013,7.390,Low Sugar,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1
3,DRB13,BABATUNJI010,DRB13_BABATUNJI010,6.115,Normal Sugar,0.011791,Soft Drinks,472.63,2005,Medium,Cluster 3,Grocery Store
4,DRB13,BABATUNJI013,DRB13_BABATUNJI013,6.115,Normal Sugar,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1
...,...,...,...,...,...,...,...,...,...,...,...,...
3527,NCZ42,BABATUNJI010,NCZ42_BABATUNJI010,10.500,Low Sugar,0.000000,Household,595.81,2005,Medium,Cluster 3,Grocery Store
3528,NCZ42,BABATUNJI013,NCZ42_BABATUNJI013,10.500,Low Sugar,0.011279,Household,592.56,1994,High,Cluster 3,Supermarket Type1
3529,NCZ42,BABATUNJI049,NCZ42_BABATUNJI049,10.500,Ultra Low Sugar,0.011305,Household,588.81,2006,Medium,Cluster 1,Supermarket Type1
3530,NCZ53,BABATUNJI010,NCZ53_BABATUNJI010,9.600,Low Sugar,0.040970,Health and Hygiene,471.80,2005,Medium,Cluster 3,Grocery Store


In [24]:
duplicates =train_df.duplicated().sum()
print("number of duplicates:", duplicates)

number of duplicates: 0


In [25]:
duplicates =test_df.duplicated().sum()
print("number of duplicates:", duplicates)

number of duplicates: 0


In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990 entries, 0 to 4989
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Item_ID              4990 non-null   object 
 1   Store_ID             4990 non-null   object 
 2   Item_Store_ID        4990 non-null   object 
 3   Item_Weight          4990 non-null   float64
 4   Item_Sugar_Content   4990 non-null   object 
 5   Item_Visibility      4990 non-null   float64
 6   Item_Type            4990 non-null   object 
 7   Item_Price           4990 non-null   float64
 8   Store_Start_Year     4990 non-null   int64  
 9   Store_Size           4990 non-null   object 
 10  Store_Location_Type  4990 non-null   object 
 11  Store_Type           4990 non-null   object 
 12  Item_Store_Returns   4990 non-null   float64
dtypes: float64(4), int64(1), object(8)
memory usage: 506.9+ KB


In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3532 entries, 0 to 3531
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Item_ID              3532 non-null   object 
 1   Store_ID             3532 non-null   object 
 2   Item_Store_ID        3532 non-null   object 
 3   Item_Weight          3532 non-null   float64
 4   Item_Sugar_Content   3532 non-null   object 
 5   Item_Visibility      3532 non-null   float64
 6   Item_Type            3532 non-null   object 
 7   Item_Price           3532 non-null   float64
 8   Store_Start_Year     3532 non-null   int64  
 9   Store_Size           3532 non-null   object 
 10  Store_Location_Type  3532 non-null   object 
 11  Store_Type           3532 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 331.3+ KB


In [28]:
label_encoder=LabelEncoder()

In [29]:
#List of categorical columns to encode
categorical_cols=['Item_ID', 'Store_ID', 'Item_Store_ID', 'Item_Sugar_Content', 'Item_Type', 'Store_Size', 'Store_Location_Type', 'Store_Type']

#Apply label encoding to each categorical column
for col in categorical_cols:
    train_df[col] = label_encoder.fit_transform(train_df[col])

In [30]:
#List of categorical columns to encode
categorical_cols=['Item_ID', 'Store_ID', 'Item_Store_ID', 'Item_Sugar_Content', 'Item_Type', 'Store_Size', 'Store_Location_Type', 'Store_Type']

#Apply label encoding to each categorical column
for col in categorical_cols:
    test_df[col] = label_encoder.fit_transform(test_df[col])

In [31]:
train_df

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
0,0,0,0,11.600000,0,0.068535,14,357.54,2005,1,2,0,709.08
1,0,1,1,11.600000,0,0.040912,14,355.79,1994,0,2,1,6381.69
2,0,2,2,11.600000,0,0.041178,14,350.79,2014,1,1,1,6381.69
3,0,3,3,11.600000,0,0.041113,14,355.04,2016,1,2,2,2127.23
4,0,6,4,11.600000,2,0.000000,14,354.79,2011,2,1,1,2481.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,1450,4,4985,12.908838,0,0.145952,9,402.39,1992,2,0,0,406.14
4986,1450,5,4986,12.908838,0,0.082956,9,410.14,1992,1,2,3,13808.69
4987,1450,7,4987,14.650000,0,0.083528,9,406.14,2009,1,1,1,5685.93
4988,1450,8,4988,14.650000,0,0.083359,9,404.89,2004,2,0,1,11778.00


In [32]:
scaler=StandardScaler()

In [33]:
train_df['Item_ID']=scaler.fit_transform(train_df[['Item_ID']])
train_df['Store_ID']=scaler.fit_transform(train_df[['Store_ID']])
train_df['Item_Store_ID']=scaler.fit_transform(train_df[['Item_Store_ID']])

In [34]:
test_df['Item_ID']=scaler.fit_transform(test_df[['Item_ID']])
test_df['Store_ID']=scaler.fit_transform(test_df[['Store_ID']])
test_df['Item_Store_ID']=scaler.fit_transform(test_df[['Item_Store_ID']])

In [35]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990 entries, 0 to 4989
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Item_ID              4990 non-null   float64
 1   Store_ID             4990 non-null   float64
 2   Item_Store_ID        4990 non-null   float64
 3   Item_Weight          4990 non-null   float64
 4   Item_Sugar_Content   4990 non-null   int32  
 5   Item_Visibility      4990 non-null   float64
 6   Item_Type            4990 non-null   int32  
 7   Item_Price           4990 non-null   float64
 8   Store_Start_Year     4990 non-null   int64  
 9   Store_Size           4990 non-null   int32  
 10  Store_Location_Type  4990 non-null   int32  
 11  Store_Type           4990 non-null   int32  
 12  Item_Store_Returns   4990 non-null   float64
dtypes: float64(7), int32(5), int64(1)
memory usage: 409.5 KB


In [36]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3532 entries, 0 to 3531
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Item_ID              3532 non-null   float64
 1   Store_ID             3532 non-null   float64
 2   Item_Store_ID        3532 non-null   float64
 3   Item_Weight          3532 non-null   float64
 4   Item_Sugar_Content   3532 non-null   int32  
 5   Item_Visibility      3532 non-null   float64
 6   Item_Type            3532 non-null   int32  
 7   Item_Price           3532 non-null   float64
 8   Store_Start_Year     3532 non-null   int64  
 9   Store_Size           3532 non-null   int32  
 10  Store_Location_Type  3532 non-null   int32  
 11  Store_Type           3532 non-null   int32  
dtypes: float64(6), int32(5), int64(1)
memory usage: 262.3 KB


In [73]:
X=train_df.drop('Item_Store_Returns', axis=1)
y=train_df['Item_Store_Returns']
X_train, X_val, y_train, y_val= train_test_split(X, y, test_size=0.1, random_state=44)

In [75]:
model = XGBRegressor()

In [77]:
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [84]:
val_preds = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, val_preds))
print("\nValidation RMSE:", rmse)


Validation RMSE: 3378.9063731853034


In [96]:
test_preds = model.predict(test_df)

In [104]:
submission_file = submission.copy()
submission_file['Item_Store_Returns'] = test_preds

In [106]:
submission_file.to_csv("submission.csv", index = False)